### Multiple Testing Correction

### Why correct for multiple tests?

Let's re-state the usual way we think about hypothesis testing, in the scenario where we don't have a specified alternative hypothesis.
  > In this case we define as *significant* any result that falls outside of our confidence interval (say 95%). 
  
Notice that this means that we expect 5% of our results to be significant by chance alone. We just decided to accept that something that unlikely is significant. 

But we are only doing one test. What if we do 100 tests? Then we expect 5 of them to be significant by chance alone.

### Error rates.

- **the family-wise error rate (FWER)** is the probability of making *at least one* false discovery.

- **the false positive** rate is the proportion of tests that are significant by chance alone. It is defined as the number of false positivers, divided by the number of false positives plus the number of true negatives. It is also called the *type I error rate*.

$$ \text{false positive rate} = \frac{\text{number of false positives}}{\text{number of false positives} + \text{number of true negatives}} $$
- **the false discovery** rate is the proportion of significant results that are false positives. It is defined as:

$$ \text{false discovery rate} = \frac{\text{number of false positives}}{\text{number of significant results}} $$


#### Example - court of law

To illustrate the difference between the last two. In a court of law, if:

- you are asking about false positive rate:
   > you are asking about the probability that an innocent person is convicted.

- you are asking about false discovery rate:
   > you are asking about the probability that a convicted person is innocent.


#### Example - Python



In [3]:
import numpy as np
import scipy.stats as stats

np.random.seed(0)

alpha = 0.05
num_tests = 100
num_positive = 0


print(f"simulation normal distribution with mean=0, std=1")
print(f"t-test with null hypothesis: mean=0")
print(f"Performing {num_tests} simulations / tests with alpha={alpha}")

for i in range(num_tests):
    data = np.random.normal(0, 1, 100)  # Generate random data
    _, p_value = stats.ttest_1samp(data, 0)  # Perform t-test
    
    if p_value < alpha:
        num_positive += 1

print(f"Number of positive results: {num_positive}")

simulation normal distribution with mean=0, std=1
t-test with null hypothesis: mean=0
Performing 100 simulations / tests with alpha=0.05
Number of positive results: 6



### Controlling the false positive rate

#### Bonferroni correction

The Bonferroni correction controls for the family-wise error rate.

Given a desivered lower bound $\alpha$ on the family-wise error rate (the $p$-value), the Bonferroni correction tests each hypothesis at a significance level of $\alpha / m$, where $m$ is the number of tests.

Procedure:
- calculate the p-value for each test
- call as significant any test with p-value less than $\alpha / m$.

Pros: Easy to understand and implement.
Cons: Can be very conservative.

#### Controlling for the false discovery rate

The Benjamini-Hochberg procedure controls for the false discovery rate.

It is less conservative than the bonferroni correction. 

Procedure:
- calculate the p-value for each test
- order the p-values from smallest to largest
- call as significant any test with p-value less than $\alpha i / m$, where $i$ is the rank of the p-value in the ordered list.

Pros: Less conservative than Bonferroni correction.
Cons: Can be difficult to understand and implement.

#### Adjusted P-values

Another method of controlling the false discovery rate is to adjust the p-values themselves.




In [11]:
import numpy as np
import scipy.stats as stats

np.random.seed(0)

alpha = 0.040
num_tests = 1000
num_positive = 0

# Lists to store the results for each correction
results_no_correction = []
results_bonferroni = []
results_benjamin_hochberg = []
results_adjusted_pvalues = []
pvalues= []

for i in range(num_tests): # notice the variable i
    data = np.random.normal(0, 1, 100)  # Generate random data
    _, p_value = stats.ttest_1samp(data, 0)  # Perform t-test
    
    results_no_correction.append(p_value < alpha)
    results_bonferroni.append(p_value < alpha / num_tests)
    pvalues.append(p_value)


results_benjamin_hochberg = np.array(sorted(pvalues)) < alpha / num_tests
results_adjusted_pvalues=  np.array(sorted(pvalues)) * num_tests < alpha

num_positive_no_correction = sum(results_no_correction)
num_positive_bonferroni = sum(results_bonferroni)
num_positive_benjamin_hochberg = sum(results_benjamin_hochberg)
num_positive_adjusted_pvalues = sum(results_adjusted_pvalues)

print(f"Number of positive results without correction: {num_positive_no_correction}")
print(f"Number of positive results with Bonferroni correction: {num_positive_bonferroni}")
print(f"Number of positive results with Benjamin-Hochberg correction: {num_positive_benjamin_hochberg}")
print(f"Number of positive results with adjusted p-values correction: {num_positive_adjusted_pvalues}")

Number of positive results without correction: 51
Number of positive results with Bonferroni correction: 0
Number of positive results with Benjamin-Hochberg correction: 0
Number of positive results with adjusted p-values correction: 0


### Notes on corrected p-values

- Corrected p-values do not behave as normal p-values. 

In some circumstances you might be brought to study the distribution of p-values. This is a bad idea if you are using corrected p-values.

Circumstances in which you might want to study the distribution of p-values:
- you are trying to understand the behaviour of a test
- you are checking for bias
- you are comparing the behaviour of different tests